In [1]:
import os
from pathlib import Path
import re
from datetime import date,datetime
from configparser import ConfigParser
import psycopg2
from psycopg2.extensions import AsIs
import pandas as pd

In [49]:
# work with xlsx workbooks
import openpyxl
from openpyxl import Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.styles import Alignment, PatternFill, Border, Font # Side, Alignment, Protection,
from openpyxl.formatting import Rule
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.comments import Comment
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter

In [2]:
import pyprojroot
repodir = pyprojroot.find_root(pyprojroot.has_dir(".git"))

In [3]:
filename = Path(os.path.expanduser('~')) / ".database.ini"
section = 'iucnecoadm'

parser = ConfigParser()
parser.read(filename)
db = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        db[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

params = db

In [4]:
conn = psycopg2.connect(**db)
cur = conn.cursor()

In [5]:
qry = """
SELECT code, biome_code, name, shortname, update, shortdesc, keyfeatures, distdesc 
FROM functional_groups 
ORDER BY biome_code ;"""
cur.execute(qry )
short_table = cur.fetchall()

In [6]:
qry = """
SELECT code, name,
et.description as traits, k.description as key_drivers, d.description as distribution, 
array_to_string(contributors,' :: '), 'Content version ' || version || ' updated ' || date(k.update)
FROM efg_key_ecological_drivers as k
LEFT JOIN efg_ecological_traits as et
    USING (code,language,version,contributors)
LEFT JOIN efg_distribution as d
    USING (code,language,version,contributors)
LEFT JOIN functional_groups
    USING(code)
WHERE language = 'en' AND version = 'v2.1';"""
cur.execute(qry )
long_table = cur.fetchall()

In [7]:
qry = """
SELECT code, ref_code, ref_cite, author_list,date,title,post_title,doi
FROM efg_references 
LEFT JOIN ref_list 
    USING (ref_code)
ORDER BY code;
"""
cur.execute(qry )
references = cur.fetchall()

In [8]:
cur.close()
conn.commit()
if conn is not None:
    conn.close()
    print('Database connection closed.')

Database connection closed.


In [9]:
short_table_df = pd.DataFrame(short_table,
                           columns=("code","biome code", "name", "short name", "update", "short description", "key features", "distribution summary"))

In [43]:
# "" % short_table_df['code']
urlstr = "https://global-ecosystems.org/explore/groups/%s"
short_table_df['url'] = short_table_df.apply(lambda row: urlstr % row['code'], axis=1)

In [44]:
short_table_df.iloc[1]['url']

'https://global-ecosystems.org/explore/groups/F1.4'

In [45]:
long_table_df = pd.DataFrame(long_table,
             columns=("code", "name", "ecosystem properties", "ecological drivers", "distribution", "contributors","update"))

In [46]:
refs_df = pd.DataFrame(references,
             columns=("code", "citation", "full reference", "author list", "date", "title", "post title","doi"))

In [47]:
refs_df.head()

,code,citation,full reference,author list,date,title,post title,doi
0,F1.1,"Meyer JL, Strayer DL, Wallace JB, Eggert SL, H...","Meyer JL, Strayer DL, Wallace JB, Eggert SL, H...","Meyer JL, Strayer DL, Wallace JB, Eggert SL, H...",2007,The contribution of headwater streams to biodi...,43: 86–103,10.1111/j.1752-1688.2007.00008.x
1,F1.1,"Meyer JL, Wallace JB 2001","Meyer JL, Wallace JB (2001) **Lost linkages an...","Meyer JL, Wallace JB",2001,None,"Blackwell, Oxford",None
2,F1.1,"Giller PS., Giller P., Malmqvist B 1998","Giller PS., Giller P., Malmqvist B (1998) **Th...","Giller PS., Giller P., Malmqvist B",1998,None,"Oxford University Press, Oxford",None
3,F1.2,"Tockner K, Malard, F, Ward JV 2000","Tockner K, Malard, F, Ward JV (2000) An extens...","Tockner K, Malard, F, Ward JV",2000,An extension of the flood pulse concept,14: 2861-2883,10.1002/1099-1085(200011/12)14:16/17<2861::aid...
4,F1.3,Olsson TI 1981,Olsson TI (1981) Overwintering of benthic macr...,Olsson TI,1981,Overwintering of benthic macroinvertebrates in...,4: 161-166,10.1111/j.1600-0587.1981.tb00993.x


In [48]:
outfile = "IUCN-GET-profiles-exported-%s.xlsx" % date.today()

Should include a tab with following info, and conditions:
Worksheet 1 includes code, name, short name and short descriptions and summaries, Worksheet 2 contains the complete text for three sections of the profile and names of profile authors, Worksheet 3 contains the list of references for each unit. Use column 'code' to join content from different worksheets. 

Also add one column with fully formatted citation and weblink to facilitate reuse.

In [50]:
cent_align=Alignment(horizontal='center', vertical='center', wrap_text=False)
wrap_align=Alignment(horizontal='left', vertical='top', wrap_text=True)

fontSmall = Font(size = "9")


sheet_colors = {"intro": "1072BA" , "summary": "5AFF5A", "default":"505050", "addentry": "20CA82"}

table_style={"Instructions":TableStyleInfo(name="TableStyleMedium9", showFirstColumn=True, showLastColumn=False, 
                                           showRowStripes=True, showColumnStripes=False),

             }

In [51]:
wb = Workbook()
ws = wb.active
item = {"title": "About", "colWidths":[("A",90),("B",40)], "tabColor":"intro","active":True}
ws.title = item['title']
for k in item['colWidths']:
    for j in k[0]:
        ws.column_dimensions[j].width = k[1]
ws.sheet_properties.tabColor = sheet_colors[item["tabColor"]]

In [52]:
info = ("Profiles of IUCN Global Ecosystem Typology Level 3 units",
        "Version 2.1 (June 2022)",
        "This data export reflects the status of the database on the %s" % date.today().strftime('%d %b %Y'),
        "Please cite the source of this work as:",
        "Keith, D. A., J. R. Ferrer-Paris, E. Nicholson, M. Bishop, B. A, Polidoro, E. Ramirez-Llodra, M. G. Tozer, J. L. Nel, R. Mac Nally, E. J. Gregr, K. E. Watermeyer, F. Essl, D. Faber-Langendoen, J. Franklin, C. E. R. Lehmann, A. Etter, D. J. Roux, J. S. Stark, J. A. Rowland, N. A. Brummitt, U. C. Fernandez-Arcaya, I. M. Suthers, S. K. Wiser, I. Donohue, L. J. Jackson, R. T. Pennington, N. Pettorelli, A. Andrade, A. Lindgaard, T. Tahvanainen, A. Terauds, M. A. Chadwick, N. J. Murray, J. Moat, P. Pliscoff, I. Zager, and R. T. Kingsford (2022) A function-based typology for Earth’s ecosystems Nature 610, 513–518. DOI:10.1038/s41586-022-05318-4.", 
        "On line at https://global-ecosystems.org/",
        "DESCRIPTION:",
        "Workbook with profile content for 110 Ecosystem Functional Groups of the IUCN Global Ecosystem Typology (Level 3 units).",
        "CONDITIONS:",
        "To reuse this content in other on-line resources you have to agree with these three conditions:",
        "1) acknowledge primary sources with citations & web links (https://global-ecosystems.org/)",
        "2) include version number and update with new versions whenever possible",
        "3) no modifications without agreement from IUCN Commission on Ecosystem Management or the relevant Working/Thematic Group",
        "CONTENT:",
        "Worksheet 1 (Short description) includes code, name, short name and short descriptions and summaries",
        "Worksheet 2 (Profile text) contains the complete text for three sections of the profile and names of profile authors",
        "Worksheet 3 (List of references) contains the list of references for each unit. ",
        "Use column 'code' to join content from different worksheets."
        )

k = 1
for row in info:
    ws.cell(k,1,value=row)
    ws.cell(k,1).alignment=wrap_align
    k=k+1
    
ws.cell(1,1).style='Title'
ws.cell(6,1).hyperlink='https://global-ecosystems.org/'
ws.cell(6,1).style='Hyperlink'

# Disclaimer
ws.cell(9,1).font=Font(color="FF0000", bold=True,italic=False) 
ws.cell(11,1).font=Font(color="FF0000", italic=True) 
ws.cell(12,1).font=Font(color="FF0000", italic=True) 
ws.cell(13,1).font=Font(color="FF0000", italic=True) 


supporters = (
    {'institution':"International Union for the Conservation of Nature",'url':"https://www.iucn.org/"},
    {'institution':"IUCN Commission on Ecosystem Management",'url':"https://www.iucn.org/commissions/commission-ecosystem-management"},
    {'institution':"IUCN Red List of Ecosystems Thematic Group",'url':"https://iucnrle.org/"},
    {'institution':"University of New South Wales",'url':"https://www.unsw.edu.au/"},
    {'institution':"UNSW Centre for Ecosystem Science",'url':"https://www.ecosystem.unsw.edu.au/"},
 )
    

k=k+2
ws.cell(k-1,1,value="Links:")
for item in supporters:
    cell=ws.cell(k,1)
    cell.value=item['institution']
    cell.hyperlink=item['url']
    cell.style = "Hyperlink"
    k=k+1


In [53]:
wb.save(repodir / 'data' / outfile)

We then add the other worksheets with pandas:

In [54]:
with pd.ExcelWriter(repodir / 'data' / outfile, mode="a") as writer:  
    short_table_df.to_excel(writer, sheet_name='Short description', index=False, freeze_panes=(1,1))
    long_table_df.to_excel(writer, sheet_name='Profile text', index=False, freeze_panes=(1,1))
    refs_df.to_excel(writer, sheet_name='List of References', index=False, freeze_panes=(1,2))